## <span style="color:darkgreen">IBM Watson AutoAI - Generated Scikit-learn Notebook v1.14.4</span>

Consider these tips for working with an auto-generated notebook:
- Notebook code generated using AutoAI will execute successfully. If you modify the notebook, we cannot guarantee it will run successfully.
- This pipeline is optimized for the original data set. The pipeline might fail or produce sub-optimum results if used with different data.  If you want to use a different data set, consider retraining the AutoAI experiment to generate a new pipeline. For more information, see <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/autoai-notebook.html">Cloud Platform</a> 
- Before modifying the pipeline or trying to re-fit the pipeline, consider that the code converts dataframes to numpy arrays before fitting the pipeline (a current restriction of the preprocessor pipeline).


<a id="content"></a>
## Notebook content

This notebook contains a Scikit-learn representation of AutoAI pipeline. This notebook introduces commands for getting data, training the model, and testing the model. 

Some familiarity with Python is helpful. This notebook uses Python 3.7 and scikit-learn 0.23.1.

## Notebook goals

-  Scikit-learn pipeline definition
-  Pipeline training 
-  Pipeline evaluation

## Contents

This notebook contains the following parts:

**[Setup](#setup)**      
&nbsp;&nbsp;[Package installation](#install)   
&nbsp;&nbsp;[AutoAI experiment metadata](#variables_definition)   
**[Pipeline inspection](#inspection)**      
&nbsp;&nbsp;[Read training data](#read)    
&nbsp;&nbsp;[Train and test data split](#split)    
&nbsp;&nbsp;[Make pipeline](#preview_model_to_python_code)      
&nbsp;&nbsp;[Train pipeline model](#train)  
&nbsp;&nbsp;[Test pipeline model](#test_model)  
**[Next steps](#next_steps)**   
**[Copyrights](#copyrights)**

<a id="setup"></a>
# Setup

<a id="install"></a>
## Package installation
Before you use the sample code in this notebook, install the following packages:
 - ibm_watson_machine_learning,
 - autoai-libs.


In [ ]:
!pip install ibm-watson-machine-learning | tail -n 1
!pip install -U autoai-libs | tail -n 1

<a id="variables_definition"></a>
## AutoAI experiment metadata
The following cell contains the training data connection details.  
**Note**: The connection might contain authorization credentials, so be careful when sharing the notebook.

In [ ]:
# @hidden_cell
from ibm_watson_machine_learning.helpers import DataConnection, S3Connection, S3Location

training_data_reference = [DataConnection(
    connection=S3Connection(
        api_key='WKdmICGCkbaP4r3sa5k1yp1iCCSj0CxJV9QqFFJySTR8',
        auth_endpoint='https://iam.bluemix.net/oidc/token/',
        endpoint_url='https://s3-api.us-geo.objectstorage.softlayer.net'
    ),
        location=S3Location(
        bucket='covid-donotdelete-pr-3exfkmimvnsc5i',
        path='viz1_df.csv'
    )),
]
training_result_reference = DataConnection(
    connection=S3Connection(
        api_key='WKdmICGCkbaP4r3sa5k1yp1iCCSj0CxJV9QqFFJySTR8',
        auth_endpoint='https://iam.bluemix.net/oidc/token/',
        endpoint_url='https://s3-api.us-geo.objectstorage.softlayer.net'
    ),
    location=S3Location(
        bucket='covid-donotdelete-pr-3exfkmimvnsc5i',
        path='auto_ml/07c72f69-c9f6-4d4f-bcca-6ac2baac40ad/wml_data/f9c6db8a-7c99-4952-a5e3-416c75d3eb80/data/automl',
        model_location='auto_ml/07c72f69-c9f6-4d4f-bcca-6ac2baac40ad/wml_data/f9c6db8a-7c99-4952-a5e3-416c75d3eb80/data/automl/cognito_output/Pipeline1/model.pickle',
        training_status='auto_ml/07c72f69-c9f6-4d4f-bcca-6ac2baac40ad/wml_data/f9c6db8a-7c99-4952-a5e3-416c75d3eb80/training-status.json'
    ))

Following cell contains input parameters provided to run the AutoAI experiment in Watson Studio.

In [ ]:
experiment_metadata = dict(
   prediction_type='classification',
   prediction_column='died',
   test_size=0.1,
   scoring='f1',
   project_id='db25f879-2c46-4936-b2ee-f9b3526ea5f5',
   space_id='None',
   deployment_url='https://us-south.ml.cloud.ibm.com',
   csv_separator=',',
   random_state=33,
   max_number_of_estimators=2,
   daub_include_only_estimators=None,
   training_data_reference=training_data_reference,
   training_result_reference=training_result_reference,
   positive_label='1.0'
)

<a id="inspection"></a>
# Pipeline inspection

<a id="read"></a>
## Read training data

Retrieve training dataset from AutoAI experiment as pandas DataFrame.

In [ ]:
df = training_data_reference[0].read(csv_separator=experiment_metadata['csv_separator'])
df.dropna('rows', how='any', subset=[experiment_metadata['prediction_column']], inplace=True)

<a id="split"></a>
##  Train and test data split

In [ ]:
from sklearn.model_selection import train_test_split

df.drop_duplicates(inplace=True)
X = df.drop([experiment_metadata['prediction_column']], axis=1).values
y = df[experiment_metadata['prediction_column']].values

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=experiment_metadata['test_size'],
                                                    stratify=y, random_state=experiment_metadata['random_state'])

<a id="preview_model_to_python_code"></a>
## Make pipeline
In the next cell, you can find the Scikit-learn definition of the selected AutoAI pipeline.

Import statements.

In [ ]:
from autoai_libs.transformers.exportable import ColumnSelector
from autoai_libs.transformers.exportable import NumpyColumnSelector
from autoai_libs.transformers.exportable import CompressStrings
from autoai_libs.transformers.exportable import NumpyReplaceMissingValues
from autoai_libs.transformers.exportable import NumpyReplaceUnknownValues
from autoai_libs.transformers.exportable import boolean2float
from autoai_libs.transformers.exportable import CatImputer
from autoai_libs.transformers.exportable import CatEncoder
import numpy as np
from autoai_libs.transformers.exportable import float32_transform
from sklearn.pipeline import make_pipeline
from autoai_libs.transformers.exportable import FloatStr2Float
from autoai_libs.transformers.exportable import NumImputer
from autoai_libs.transformers.exportable import OptStandardScaler
from sklearn.pipeline import make_union
from autoai_libs.transformers.exportable import NumpyPermuteArray
from autoai_libs.cognito.transforms.transform_utils import TA1
import autoai_libs.utils.fc_methods
from autoai_libs.cognito.transforms.transform_utils import FS1
from autoai_libs.cognito.transforms.transform_utils import TAM
from sklearn.cluster import FeatureAgglomeration
from sklearn.linear_model import LogisticRegression

#### Pre-processing & Estimator.

In [ ]:
column_selector = ColumnSelector(
    columns_indices_list=[
        0,
        4,
        5,
        6,
        7,
        8,
        9,
        10,
        11,
        12,
        13,
        14,
        15,
        16,
        17,
        18,
        19,
        20,
        21,
    ]
)
numpy_column_selector_0 = NumpyColumnSelector(
    columns=[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
)
compress_strings = CompressStrings(
    compress_type="hash",
    dtypes_list=[
        "char_str",
        "int_num",
        "char_str",
        "char_str",
        "char_str",
        "char_str",
        "char_str",
        "char_str",
        "char_str",
        "char_str",
        "char_str",
        "char_str",
        "char_str",
        "char_str",
        "char_str",
        "char_str",
        "char_str",
        "char_str",
    ],
    missing_values_reference_list=["", "-", "?", float("nan")],
    misslist_list=[
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
    ],
)
numpy_replace_missing_values_0 = NumpyReplaceMissingValues(
    missing_values=[], filling_values=float("nan")
)
numpy_replace_unknown_values = NumpyReplaceUnknownValues(
    filling_values=float("nan"),
    filling_values_list=[
        float("nan"),
        float("nan"),
        float("nan"),
        float("nan"),
        float("nan"),
        float("nan"),
        float("nan"),
        float("nan"),
        float("nan"),
        float("nan"),
        float("nan"),
        float("nan"),
        float("nan"),
        float("nan"),
        float("nan"),
        float("nan"),
        float("nan"),
        float("nan"),
    ],
    known_values_list=[
        [
            243382710732531489693195870365903964183,
            132302907946296948163998721551176913917,
        ],
        [
            0,
            1,
            2,
            3,
            4,
            5,
            6,
            7,
            8,
            9,
            10,
            11,
            12,
            13,
            14,
            15,
            16,
            17,
            18,
            33,
            35,
        ],
        [
            248552393553625911829275175950507086347,
            196453806044956024215798110170812425269,
        ],
        [
            248552393553625911829275175950507086347,
            196453806044956024215798110170812425269,
        ],
        [
            248552393553625911829275175950507086347,
            196453806044956024215798110170812425269,
        ],
        [
            248552393553625911829275175950507086347,
            196453806044956024215798110170812425269,
        ],
        [
            248552393553625911829275175950507086347,
            196453806044956024215798110170812425269,
        ],
        [
            248552393553625911829275175950507086347,
            196453806044956024215798110170812425269,
        ],
        [
            248552393553625911829275175950507086347,
            196453806044956024215798110170812425269,
        ],
        [
            248552393553625911829275175950507086347,
            196453806044956024215798110170812425269,
        ],
        [
            248552393553625911829275175950507086347,
            196453806044956024215798110170812425269,
        ],
        [
            248552393553625911829275175950507086347,
            196453806044956024215798110170812425269,
        ],
        [
            248552393553625911829275175950507086347,
            196453806044956024215798110170812425269,
        ],
        [
            248552393553625911829275175950507086347,
            196453806044956024215798110170812425269,
        ],
        [
            248552393553625911829275175950507086347,
            196453806044956024215798110170812425269,
        ],
        [
            248552393553625911829275175950507086347,
            196453806044956024215798110170812425269,
        ],
        [
            67173585239538071839587539288433045831,
            182244586587704005283142733706507394948,
        ],
        [
            248552393553625911829275175950507086347,
            196453806044956024215798110170812425269,
        ],
    ],
    missing_values_reference_list=["", "-", "?", float("nan")],
)
cat_imputer = CatImputer(
    strategy="most_frequent",
    missing_values=float("nan"),
    sklearn_version_family="23",
)
cat_encoder = CatEncoder(
    encoding="ordinal",
    categories="auto",
    dtype=np.float64,
    handle_unknown="error",
    sklearn_version_family="23",
)
pipeline_0 = make_pipeline(
    numpy_column_selector_0,
    compress_strings,
    numpy_replace_missing_values_0,
    numpy_replace_unknown_values,
    boolean2float(),
    cat_imputer,
    cat_encoder,
    float32_transform(),
)
numpy_column_selector_1 = NumpyColumnSelector(columns=[1])
float_str2_float = FloatStr2Float(
    dtypes_list=["int_num"], missing_values_reference_list=[]
)
numpy_replace_missing_values_1 = NumpyReplaceMissingValues(
    missing_values=[], filling_values=float("nan")
)
num_imputer = NumImputer(strategy="median", missing_values=float("nan"))
opt_standard_scaler = OptStandardScaler(
    num_scaler_copy=None,
    num_scaler_with_mean=None,
    num_scaler_with_std=None,
    use_scaler_flag=False,
)
pipeline_1 = make_pipeline(
    numpy_column_selector_1,
    float_str2_float,
    numpy_replace_missing_values_1,
    num_imputer,
    opt_standard_scaler,
    float32_transform(),
)
union = make_union(pipeline_0, pipeline_1)
numpy_permute_array = NumpyPermuteArray(
    axis=0,
    permutation_indices=[
        0,
        2,
        3,
        4,
        5,
        6,
        7,
        8,
        9,
        10,
        11,
        12,
        13,
        14,
        15,
        16,
        17,
        18,
        1,
    ],
)
ta1 = TA1(
    fun=np.rint,
    name="round",
    datatypes=["numeric"],
    feat_constraints=[autoai_libs.utils.fc_methods.is_not_categorical],
    col_names=[
        "sex",
        "age",
        "entry_symptoms_delta",
        "intubed",
        "pneumonia",
        "pregnancy",
        "diabetes",
        "copd",
        "asthma",
        "inmsupr",
        "hypertension",
        "other_disease",
        "cardiovascular",
        "obesity",
        "renal_chronic",
        "tobacco",
        "contact_other_covid",
        "Test_result",
        "icu",
    ],
    col_dtypes=[
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
    ],
)
fs1_0 = FS1(
    cols_ids_must_keep=range(0, 19),
    additional_col_count_to_keep=15,
    ptype="classification",
)
tam = TAM(
    tans_class=FeatureAgglomeration(),
    name="featureagglomeration",
    col_names=[
        "sex",
        "age",
        "entry_symptoms_delta",
        "intubed",
        "pneumonia",
        "pregnancy",
        "diabetes",
        "copd",
        "asthma",
        "inmsupr",
        "hypertension",
        "other_disease",
        "cardiovascular",
        "obesity",
        "renal_chronic",
        "tobacco",
        "contact_other_covid",
        "Test_result",
        "icu",
        "round(age)",
        "round(entry_symptoms_delta)",
    ],
    col_dtypes=[
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
    ],
)
fs1_1 = FS1(
    cols_ids_must_keep=range(0, 19),
    additional_col_count_to_keep=15,
    ptype="classification",
)
logistic_regression = LogisticRegression(
    class_weight="balanced",
    intercept_scaling=0.004904450517961179,
    max_iter=164,
    multi_class="ovr",
    n_jobs=1,
    random_state=33,
    solver="liblinear",
    tol=0.00019584720174044179,
)


Pipeline.

In [ ]:
pipeline = make_pipeline(
    column_selector,
    union,
    numpy_permute_array,
    ta1,
    fs1_0,
    tam,
    fs1_1,
    logistic_regression,
)

<a id="train"></a>
## Train pipeline model


### Define scorer from the optimization metric
This cell constructs the cell scorer based on the experiment metadata.

In [ ]:
from sklearn.metrics import get_scorer, make_scorer

scorer = get_scorer(experiment_metadata['scoring'])
kwargs = {'pos_label':'1.0'}
scorer = make_scorer(scorer._score_func, **kwargs)

<a id="test_model"></a>
### Fit pipeline model
In this cell, the pipeline is fitted.

In [ ]:
pipeline.fit(train_X,train_y)

<a id="test_model"></a>
## Test pipeline model

Score the fitted pipeline with the generated scorer using the holdout dataset.

In [ ]:
score = scorer(pipeline, test_X, test_y)
print(score)

<a id="next_steps"></a>
# Next steps

#### [Model deployment as webservice](https://github.com/IBM/watson-machine-learning-samples/tree/master/cloud/notebooks/python_sdk/deployments/autoai)
#### [Run AutoAI experiment with python SDK](https://github.com/IBM/watson-machine-learning-samples/tree/master/cloud/notebooks/python_sdk/experiments/autoai)  

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2020 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs  
(or equivalent) and License Information document for Watson Studio Auto-generated Notebook (License Terms),  
such agreements located in the link below. Specifically, the Source Components and Sample Materials clause  
included in the License Information document for Watson Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="http://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BHU2B7&title=IBM%20Watson%20Studio%20Auto-generated%20Notebook%20V2.1">License Terms</a>  

___